In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


EXP 2 : E vs S (data brute)

In [2]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 14.8 MB/s eta 0:00:00


No options

In [ ]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)
            flattened_coeffs = np.hstack(coeffs)  # Flatten coefficients into a 1D array

            data.append(flattened_coeffs)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile( loss='binary_crossentropy', metrics=['accuracy'])

    # Start the timer
    start_time = time.time()

    # Step 5: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 6: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 7: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 8: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 9: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


    # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp2/No_Options.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.5245 - loss: 75.3273 - val_accuracy: 0.7500 - val_loss: 29.0235
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9252 - loss: 6.3102 - val_accuracy: 0.8167 - val_loss: 8.7818
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9831 - loss: 0.9253 - val_accuracy: 0.7167 - val_loss: 12.1380
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.7167 - val_loss: 11.9937
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 4.4968e-05 - val_accuracy: 0.7167 - val_loss: 11.9832
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 6.1141e-05 - val_accuracy: 0.7167 - val_loss: 11.9732
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 2.3792e-05 - val_accuracy: 0.7167 - val_loss: 11.9598
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 3.2778e-05 - val_accuracy: 0.7167 - val_l

with standerscaler

In [ ]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)
            flattened_coeffs = np.hstack(coeffs)  # Flatten coefficients into a 1D array

            data.append(flattened_coeffs)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Standardize the data using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Step 5: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile( loss='binary_crossentropy', metrics=['accuracy'])

    # Step 6: Start the timer
    start_time = time.time()

    # Step 7: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 8: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 9: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 10: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 11: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - accuracy: 0.6055 - loss: 0.5085 - val_accuracy: 0.9167 - val_loss: 0.2369
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9963 - loss: 0.0658 - val_accuracy: 0.8833 - val_loss: 0.3123
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0073 - val_accuracy: 0.8833 - val_loss: 0.4102
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.8667 - val_loss: 0.4859
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.8500 - val_loss: 0.5480
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 9.9378e-04 - val_accuracy: 0.8500 - val_loss: 0.5962
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 6.8366e-04 - val_accuracy: 0.8500 - val_loss: 0.6491
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 1.0000 - loss: 5.0112e-04 - val_accuracy: 0.8500 - val_loss: 0.689

with Optimzer Adam

In [ ]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)
            flattened_coeffs = np.hstack(coeffs)  # Flatten coefficients into a 1D array

            data.append(flattened_coeffs)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Step 5: Start the timer
    start_time = time.time()

    # Step 6: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 7: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 8: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 9: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 10: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.4822 - loss: 55.2770 - val_accuracy: 0.6000 - val_loss: 26.1677
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8712 - loss: 2.7575 - val_accuracy: 0.7333 - val_loss: 28.5187
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9369 - loss: 7.7483 - val_accuracy: 0.8333 - val_loss: 9.6724
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9920 - loss: 0.1786 - val_accuracy: 0.8000 - val_loss: 7.5347
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9597 - loss: 0.3909 - val_accuracy: 0.8167 - val_loss: 7.7700
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 0.8167 - val_loss: 6.4454
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 8.5700e-05 - val_accuracy: 0.8333 - val_loss: 6.8399
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 1.9703e-05 - val_accuracy: 0.8667 - val_loss: 7.3083


EXP 2 N (epileptic) vs S (healthy) with Optimizer and Scaler

In [ ]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)
            flattened_coeffs = np.hstack(coeffs)  # Flatten coefficients into a 1D array

            data.append(flattened_coeffs)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Standardize the data using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Step 5: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Step 6: Start the timer
    start_time = time.time()

    # Step 7: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 8: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 9: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 10: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 11: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.7777 - loss: 0.5360 - val_accuracy: 0.9333 - val_loss: 0.1897
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 0.0560 - val_accuracy: 0.8667 - val_loss: 0.4802
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.8167 - val_loss: 0.9942
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 2.0161e-04 - val_accuracy: 0.8000 - val_loss: 1.4004
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 2.6467e-05 - val_accuracy: 0.7833 - val_loss: 1.6860
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 1.2167e-05 - val_accuracy: 0.7833 - val_loss: 1.8720
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 5.1810e-06 - val_accuracy: 0.7833 - val_loss: 1.9906
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 3.1257e-06 - val_accuracy: 0.7833 - val_los